# Data Validation

In [ ]:
# Read in GI01SUMO-00002 start and end CTD files
# CTD cast AT30-013 from Irminger 2 cruise

# CTD cast AR0701-010 from Irminger 3 cruise


# Download data via M2M

In [2]:
# First, import the Python libraries we will need
import requests
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# API Information
USERNAME ='OOIAPI-B3XI9P6PELFA9I'  # for SNW
TOKEN= '95QOG88B1NS'               # for SNW
API_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'

In [9]:
# Instrument Information
beginDT = '2015-08-18T00:00:01.500Z' 
endDT = '2015-08-20T00:00:01.500Z'

site = 'GI03FLMA'
node = 'RIM01'
instrument_G = '02-CTDMOG0'
instrument_H = '02-CTDMOH0'
method = 'recovered_inst'
stream = 'ctdmo_ghqr_instrument_recovered'

iid = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]

for i in iid:
    if i < 49:
        instrument = ''.join((instrument_G,str(i)))
        data_request_url ='/'.join((API_BASE_URL,site,node,instrument,method,stream))
        print (data_request_url)
    else:
        instrument = ''.join((instrument_H,str(i)))
        data_request_url ='/'.join((API_BASE_URL,site,node,instrument,method,stream))
        print (data_request_url)


params = {
  'beginDT':beginDT,
  'endDT':endDT,   
  'limit':1000,   
}
# https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/GI03FLMA/RIM01/02-CTDMOG040/metadata

https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG040/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG041/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG042/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG043/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG044/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG045/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv//GI03FLMA/RIM01/02-CTDMOG046/recovered_inst/ctdmo_ghqr_instrument_recovered
https://ooinet.oceanobserva

In [ ]:
# Grab the data
r = requests.get(data_request_url, params=params, auth=(USERNAME, TOKEN))
data = r.json()